In [ ]:
import pandas as pd
import pickle
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the data from the pickled file
file_path = '/content/drive/My Drive/synthetic_dataset/goal_set.p'
with open(file_path, 'rb') as file:
    consultation_data = pickle.load(file)

test_data = consultation_data['test']


In [ ]:
# Extract explicit and implicit symptoms along with their disease tags
disease_tags = []
explicit_symptoms_list = []
implicit_symptoms_list = []

for item in test_data:
    disease_tags.append(item['disease_tag'])
    explicit_symptoms = list(item['goal']['explicit_inform_slots'].keys())
    implicit_symptoms = list(item['goal']['implicit_inform_slots'].keys())
    explicit_symptoms_list.append(", ".join(explicit_symptoms))
    implicit_symptoms_list.append(", ".join(implicit_symptoms))

# Create DataFrame
df = pd.DataFrame({
    'Disease Tag': disease_tags,
    'Explicit Symptoms': explicit_symptoms_list,
    'Implicit Symptoms': implicit_symptoms_list
})

# Display the DataFrame
print(df.head(10))


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score

# Define NDCG calculation functions
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

# Prepare the data
X = df['Explicit Symptoms'] + " " + df['Implicit Symptoms']
y = df['Disease Tag']

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text data with TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Using bigrams
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train Random Forest model with hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Predict and calculate accuracy
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized Random Forest Accuracy: {accuracy:.2f}")

# Calculate NDCG for the model
y_test_bin = MultiLabelBinarizer().fit_transform([[label] for label in y_test])
y_score = best_model.predict_proba(X_test)

ndcg_scores = [ndcg_score(y_test_bin[i], y_score[i], k=10) for i in range(len(y_test))]
mean_ndcg = np.mean(ndcg_scores)
print(f"Random Forest NDCG@10: {mean_ndcg:.5f}")



In [ ]:
# Symptom matching and disease prediction logic
explicit_input = input("Enter explicit symptoms (comma-separated): ").strip().lower()
input_explicit_symptoms = [symptom.strip() for symptom in explicit_input.split(',')]
matching_symptoms = {}

# Find all implicit symptoms associated with the input explicit symptoms
for item in test_data:
    explicit_info = item.get('goal', {}).get('explicit_inform_slots', {})
    implicit_info = item.get('goal', {}).get('implicit_inform_slots', {})
    disease_tag = item['disease_tag']
    if any(symptom.lower() in [key.lower() for key in explicit_info] for symptom in input_explicit_symptoms):
        matching_symptoms[disease_tag] = matching_symptoms.get(disease_tag, set())
        matching_symptoms[disease_tag].update(implicit_info.keys())

# Store matching implicit symptoms along with user's explicit symptoms for each disease in the 'callme' memory
if matching_symptoms:
    for disease_tag, implicit_symptoms in matching_symptoms.items():
        for symptom in implicit_symptoms:
            callme[(disease_tag, symptom)] = input_explicit_symptoms
            # Add the user's input symptoms to the set
            user_input_symptoms.update(input_explicit_symptoms)

# Create a dictionary to store the user input symptoms only if it's not already added
combined_symptoms = {}
if user_input_symptoms:
    combined_symptoms['User Input Symptoms'] = list(user_input_symptoms)

# Print the results
if callme:
    for (disease_tag, symptom), explicit_symptoms in callme.items():
        print(f"Disease: {disease_tag}, Symptom: {symptom}, Explicit Symptoms: {', '.join(explicit_symptoms)}")
else:
    print("No matching implicit symptoms found.")

# Print the user input symptoms
if user_input_symptoms:
    print("User Input Symptoms:", ', '.join(combined_symptoms['User Input Symptoms']))
else:
    print("No user input symptoms provided.")

In [ ]:

# Convert the data into a DataFrame
data = {
    'disease': [],
    'symptom': [],
    'score': []
}

for disease, info in disease_data.items():
    for symptom, score in info['symptom'].items():
        data['disease'].append(disease)
        data['symptom'].append(symptom)
        data['score'].append(score)

df = pd.DataFrame(data)

# Function to find and print scores based on input
def find_scores(disease_name, symptom_name):
    # Find scores based on input
    score_df = df[(df['disease'] == disease_name) & (df['symptom'] == symptom_name)]

    # Sort scores from highest to lowest
    sorted_score_df = score_df.sort_values(by='score', ascending=False)

    # Prepare list to store results
    results = []

    # Append results to the list
    if not sorted_score_df.empty:
        for idx, row in sorted_score_df.iterrows():
            results.append((row['symptom'], row['score']))
    else:
        results.append((symptom_name, 'No scores found'))

    return results

# Function to find and print matching implicit symptoms based on explicit symptoms
def find_matching_implicit_symptoms(explicit_symptoms):
    # Matching implicit symptoms
    matching_symptoms = set()

    # Check for partial matching implicit symptoms
    for item in df.itertuples():
        for explicit_symptom in explicit_symptoms:
            explicit_words = explicit_symptom.lower().split()
            implicit_words = item.symptom.lower().split()
            # Check if all explicit words are present in implicit symptom
            if all(word in implicit_words for word in explicit_words) and \
               all(word in explicit_words for word in implicit_words):
                matching_symptoms.add(item.symptom)

    return matching_symptoms

# Process each input from the callme memory and store results
all_results = []
for (disease_name, symptom_name), explicit_symptoms in callme.items():
    all_results.extend(find_scores(disease_name, symptom_name))

# Sort all results by score in descending order, while handling 'No scores found'
all_results.sort(key=lambda x: float(x[1]) if isinstance(x[1], float) else 0, reverse=True)

# Filter out results with 'No scores found'
filtered_results = [result for result in all_results if result[1] != 'No scores found']

# Print only the first 10 results with order number
for idx, (symptom, score) in enumerate(filtered_results[:10], start=1):
    print(f"{idx}. {symptom}, {score}")

# Initialize a set to store the matched implicit symptoms
matching_implicit_symptoms = set()

# Initialize a set to store the matched implicit symptoms
matching_implicit_symptoms = set()

# Take user input for implicit symptoms
implicit_input = input("Enter implicit symptoms you have (comma-separated): ")

# Split the input into individual implicit symptoms
implicit_symptoms = [symptom.strip() for symptom in implicit_input.split(',')]

# Find matching implicit symptoms based on user input
matching_implicit_symptoms = find_matching_implicit_symptoms(implicit_symptoms)

# Check if any matching implicit symptoms are found
if matching_implicit_symptoms:
    print("\nMatched Implicit Symptoms:")
    for symptom in matching_implicit_symptoms:
        print(f"- {symptom}")

    # Update the combined_symptoms dictionary to include the matched implicit symptoms
    if 'Matched Implicit Symptoms' in combined_symptoms:
        combined_symptoms['Matched Implicit Symptoms'].update(matching_implicit_symptoms)
    else:
        combined_symptoms['Matched Implicit Symptoms'] = matching_implicit_symptoms
else:
    print("No matching implicit symptoms found.")

